In [1]:
import pandas as pd
import random
from datetime import datetime, timedelta

In [2]:
random.seed(42)

In [3]:
file_path = "C:\\Users\\Tisha\\OneDrive\\Desktop\\models\\data\\Car-----Table.xlsx"

In [4]:
cars_df = pd.read_excel(file_path,engine="openpyxl")

In [5]:
num_rentals = 5000


In [6]:
start_date = datetime(2025, 3, 26, 0, 0, 0)
end_date = datetime(2025, 4, 30, 23, 59, 59)

In [7]:
# Dictionary to track car bookings to prevent double bookings
car_bookings = {}


In [8]:
# Generate rental data
rental_data = []
rented_car_ids = set()  # Track rented cars

for rental_id in range(1, num_rentals + 1):
    # Generate a random Pickup Location (City)
    pickup_location = random.choice(cars_df["City"].unique().tolist())

    # Filter cars available in that city
    available_cars = cars_df[cars_df["City"] == pickup_location]["Car_Id"].tolist()
    
    if not available_cars:
        continue  # Skip if no cars are available for this city

    # Shuffle available cars for randomness
    random.shuffle(available_cars)

    # Try to find an available car
    car_id = None
    rental_start_date = None
    return_date = None
    
    for candidate_car_id in available_cars:
        # Generate a random Rental Date within the specified range
        random_days = random.randint(0, (end_date - start_date).days)  # Random day within range
        random_hours = random.randint(0, 23)  # Random hour
        random_minutes = random.randint(0, 59)  # Random minute
        
        rental_start_date = start_date + timedelta(days=random_days, hours=random_hours, minutes=random_minutes)

        # Generate a random rental duration (between 1 hour to 7 days = 168 hours)
        rental_duration_hours = random.randint(1, 168)
        
        # Calculate return date
        return_date = rental_start_date + timedelta(hours=rental_duration_hours)

        # Check if car is already booked during this time
        if candidate_car_id in car_bookings:
            overlapping = any(
                not (return_date <= prev_rental["Rental_Date"] or rental_start_date >= prev_rental["Return_Date"])
                for prev_rental in car_bookings[candidate_car_id]
            )
            if overlapping:
                continue  # Skip this car as it is already booked

        # Car is available, assign it
        car_id = candidate_car_id
        break

    if not car_id:
        continue  # If no available car found, skip this rental

    # Get Car's Price per Hour
    price_per_hour = cars_df.loc[cars_df["Car_Id"] == car_id, "Price per Hour (INR)"].values[0]
    base_fare = cars_df.loc[cars_df["Car_Id"] == car_id, "Base_Fare"].values[0]
    
    # Calculate total amount
    total_amount = round(base_fare + (price_per_hour * rental_duration_hours), 2)

    # Store rental details
    rental_data.append([
        rental_id, car_id, random.randint(1000, 9999), pickup_location,
        rental_start_date.strftime("%Y-%m-%d %H:%M:%S"), return_date.strftime("%Y-%m-%d %H:%M:%S"),
        rental_duration_hours, total_amount
    ])

    # Update car booking record to avoid double booking
    if car_id not in car_bookings:
        car_bookings[car_id] = []
    car_bookings[car_id].append({"Rental_Date": rental_start_date, "Return_Date": return_date})

    # Track rented car
    rented_car_ids.add(car_id)

# Create DataFrame
rentals_df = pd.DataFrame(rental_data, columns=[
    "Rental_ID", "Car_ID", "User_ID", "Pickup_Location", 
    "Rental_Date", "Return_Date", "Duration_Hours", "Total_Amount"
])

In [9]:
rentals_df

,Rental_ID,Car_ID,User_ID,Pickup_Location,Rental_Date,Return_Date,Duration_Hours,Total_Amount
0,1,8067,6107,Thiruvananthapuram,2025-04-08 17:48:00,2025-04-10 21:48:00,52,14668
1,2,5134,1158,Dehradun,2025-04-30 22:27:00,2025-05-06 22:27:00,144,52724
2,3,1445,4841,Pune,2025-04-12 14:04:00,2025-04-15 16:04:00,74,24064
3,4,3493,6016,Mopa,2025-04-13 07:25:00,2025-04-15 22:25:00,63,22377
4,5,8504,9431,Surat,2025-04-11 10:07:00,2025-04-15 18:07:00,104,21510
...,...,...,...,...,...,...,...,...
4995,4996,1414,5147,Pune,2025-03-28 22:58:00,2025-04-01 15:58:00,89,31644
4996,4997,1612,3873,Vijayawada,2025-04-05 05:57:00,2025-04-10 14:57:00,129,32068
4997,4998,1060,4836,Khowai,2025-04-30 20:40:00,2025-05-05 14:40:00,114,31830
4998,4999,8715,7325,Udaipur,2025-04-14 23:15:00,2025-04-20 01:15:00,122,32278


In [10]:
# Count unique Car_ID occurrences
car_counts = rentals_df["Car_ID"].value_counts()

# Display the result
print(car_counts)


Car_ID
583     4
5452    4
4190    4
6531    4
6786    4
       ..
88      1
7937    1
9182    1
8233    1
2625    1
Name: count, Length: 4128, dtype: int64


In [11]:
# Group by Pickup_Location and rental_date and check for duplicate car_id entries
duplicates = rentals_df.duplicated(subset=['Pickup_Location', 'Rental_Date', 'Car_ID'], keep=False)

# Display duplicate records
duplicate_entries = rentals_df[duplicates]

# Print the number of duplicates found
print(f"Number of duplicate (Pickup_Location, Rental_Date, Car_ID) entries: {duplicate_entries.shape[0]}")
print(duplicate_entries.head())  # Show a few duplicate entries

Number of duplicate (Pickup_Location, Rental_Date, Car_ID) entries: 0
Empty DataFrame
Columns: [Rental_ID, Car_ID, User_ID, Pickup_Location, Rental_Date, Return_Date, Duration_Hours, Total_Amount]
Index: []


In [12]:
# Merge the dataframes
cars_df.rename(columns={"Car_Id": "Car_ID"}, inplace=True)

merged_df = rentals_df.merge(cars_df[['Car_ID', 'City']], on="Car_ID", how="left")

# Count mismatched cases
mismatch_count = (merged_df["Pickup_Location"] != merged_df["City"]).sum()

print(f"🚨 Number of mismatched rentals: {mismatch_count}")


🚨 Number of mismatched rentals: 0


In [13]:
rentals_df.to_excel("future_cardata.xlsx", index=False)


In [14]:
# now available cars 
 #Convert Rental Dates to datetime format
rentals_df["Return_Date"] = pd.to_datetime(rentals_df["Return_Date"], format="%Y-%m-%d %H:%M:%S")

In [82]:
# Ask user for recommended Car IDs (comma-separated input)
user_input = input("Enter recommended Car IDs (comma-separated): ")
recommended_car_ids = [int(car_id.strip()) for car_id in user_input.split(",") if car_id.strip().isdigit()]

# Ask user for rental date & time
user_rental_input = input("Enter your rental date & time (YYYY-MM-DD HH:MM:SS): ").strip()
user_rental_datetime = datetime.strptime(user_rental_input, "%Y-%m-%d %H:%M:%S")

# Initialize an empty list for available cars
available_cars = []

# Step 1: Loop through each recommended Car_ID and check availability
for car_id in recommended_car_ids:
    # Get car details from Cars_Table
    car = cars_df[cars_df["Car_ID"] == car_id]

    # Skip if Car_ID is not in cars_df (handles cases where the ID is missing)
    if car.empty:
        print(f"Car ID {car_id} not found in database. Skipping...")
        continue
    
    # Get all rentals for this car
    car_rentals = rentals_df[rentals_df["Car_ID"] == car_id]

    # If car was never rented, it's available
    if car_rentals.empty:
        available_cars.append(car)
    else:
        # Get latest return datetime of this car
        last_return_datetime = car_rentals["Return_Date"].max()

        # if last_return_datetime.replace(second=0, microsecond=0) < user_rental_datetime.replace(second=0, microsecond=0):
    


        # If the latest return datetime is strictly before the user's rental datetime, add to available list
    if last_return_datetime < user_rental_datetime:
            available_cars.append(car)

# Convert available cars back to DataFrame
available_cars_df = pd.concat(available_cars, ignore_index=True).drop_duplicates() if available_cars else pd.DataFrame()





Enter recommended Car IDs (comma-separated):  1442
Enter your rental date & time (YYYY-MM-DD HH:MM:SS):  2025-04-15 16:04:20	


In [58]:
car_rentals

,Rental_ID,Car_ID,User_ID,Pickup_Location,Rental_Date,Return_Date,Duration_Hours,Total_Amount
2,3,1445,4841,Pune,2025-04-12 14:04:00,2025-04-15 16:04:00,74,24064


In [84]:
# Display available cars
print("\nAvailable Cars:")
print(available_cars_df if not available_cars_df.empty else "No available cars based on your criteria.")


Available Cars:
   Car_ID        Make              Model Car Type  Mileage (km/l)  \
0    1442  Volkswagen  Volkswagen Taigun      SUV              16   

   Year of Manufacture  Price per Hour (INR)  Occupancy   Fuel Policy   AC  \
0                 2023                   242          5  Full to Full  Yes   

  Transmission  Luggage Capacity  City         Agency_Name  Base_Fare  \
0       Manual                 2  Pune  Metro Rentals Pune       1050   

   LocationID Unlimited Mileage Free Cancellation   Ratings  
0           8               Yes                Yes      3.4  


In [74]:
cars_df[cars_df['Car_ID']==1442]

,Car_ID,Make,Model,Car Type,Mileage (km/l),Year of Manufacture,Price per Hour (INR),Occupancy,Fuel Policy,AC,Transmission,Luggage Capacity,City,Agency_Name,Base_Fare,LocationID,Unlimited Mileage,Free Cancellation,Ratings
1441,1442,Volkswagen,Volkswagen Taigun,SUV,16,2023,242,5,Full to Full,Yes,Manual,2,Pune,Metro Rentals Pune,1050,8,Yes,Yes,3.4
